In [1]:
import pandas as pd

df_LEAKAGE_181 = pd.read_excel('../dataset/Leakage variables.for Last oneand year.xlsx', skiprows=4, sheet_name='DIHD181')
df_LEAKAGE_182 = pd.read_excel('../dataset/Leakage variables.for Last oneand year.xlsx', skiprows=4, sheet_name='DIHD182')
df_LEAKAGE_183 = pd.read_excel('../dataset/Leakage variables.for Last oneand year.xlsx', skiprows=4, sheet_name='DIHD183')


In [2]:
#df_LEAKAGE = pd.concat([df_LEAKAGE_181,df_LEAKAGE_182,df_LEAKAGE_183],ignore_index=True)
df_LEAKAGE = df_LEAKAGE_182.copy()
df_LEAKAGE['Time'] = pd.to_datetime(df_LEAKAGE['Time'])
df_LEAKAGE.dropna(axis=0, subset=['Time'],inplace=True)


In [3]:
df_UT = pd.DataFrame()

df_UT['Time'] = list(df_LEAKAGE['Time'].unique())
df_UT.sort_values('Time', inplace=True)

In [4]:
temp_list_desc = []
temp_product = []
for ind_date in list(df_UT['Time']):
    temp_product.append(df_LEAKAGE[(df_LEAKAGE['Time'] == ind_date)]['Product'].unique()[0])


In [5]:
df_UT['Product Description'] = temp_product

In [6]:
def desc_finder(row):
    while row['Product Description'] != None:
        if row['Product Description'].find('HL2') != -1:
            return "HL2"
        elif row['Product Description'].find('DLITE') != -1:
            return "DLITE"
        elif row['Product Description'].find('HL1') != -1:
            return "HL1"
        elif row['Product Description'].find('HOLI') != -1:
            return "HOLI"
        elif row['Product Description'].find('DENIM8') != -1:
            return "DENIM8"
        elif row['Product Description'].find('HOLI2') != -1:
            return "HOLI2"
        elif row['Product Description'].find('DL2') != -1:
            return "DL2"
        else:
            return None

df_UT['Description'] = df_UT.apply(desc_finder, axis=1)


In [7]:
def prodDescToSize(row):
    # print(type(row['Product']))
    prodDesc = row['Product Description'].split()
    
    if prodDesc[2][0:2] == 'SM':
        return 1
    elif prodDesc[2][0:2] == 'MD':
        return 3
    elif prodDesc[2][0:2] == 'LG':
        return 4
    else:
        return 5
df_UT['Size'] = df_UT.apply(prodDescToSize, axis=1)
df_UT['Size'] = pd.to_numeric(df_UT['Size'], errors='coerce')

In [8]:
df_UT['Prod_Line'] = int(list(df_LEAKAGE['Line'].unique())[0][-1])

In [9]:
var_desc = list(df_LEAKAGE['Variable Description'].unique())

In [10]:
for var in var_desc:
    df_UT = pd.merge(df_UT,df_LEAKAGE[df_LEAKAGE['Variable Description'] == var],how='left',on='Time')
    df_UT.drop(columns=['Variable Description','Line', 'Entry On', 'Product', 'Product Group', 'Process Order', 'User', 'Production Event (Sample Id)', 'Comments'],inplace=True)
    df_UT.rename(columns={'Value': 'Value_'+var, 'LSL': 'LSL_'+var, 'LCL': 'LCL_'+var, 'Target': 'Target_'+var, 'UCL': 'UCL_'+var, 'USL': 'USL_'+var}, inplace=True)
    df_UT.sort_values('Time', inplace=True)

In [11]:
# for var in var_desc:
#     df_UT = pd.merge(df_UT,df_LEAKAGE[df_LEAKAGE['Variable Description'] == var],how='left',on='Time')
#     df_UT.drop(columns=['Variable Description','Line', 'Entry On', 'Product', 'Product Group', 'Process Order', 'User', 'Production Event (Sample Id)', 'Comments','LSL','LCL','USL','UCL','Target'],inplace=True)
#     df_UT.rename(columns={'Value': var}, inplace=True)
#     df_UT.sort_values('Time', inplace=True)

In [12]:
df_UT.fillna(method="ffill",inplace=True)
df_UT.fillna(method="bfill",inplace=True)


In [13]:
df_UT['Date'] = df_UT['Time'].dt.date

In [14]:
df_BFC = pd.read_excel('../dataset/BFC Consumer Comments MAIN_BFC CC Listing - 2021-12-14T152352.357.xlsx')
df_BFC.shape

(2381, 50)

In [15]:
def prodLine(row):
    if len(row['Production Code']) > 10:
        if row['Production Code'] == '0345B09503511':
            return 0
        else:
            return 1
    return row['Production Code'][-2]
def sizeCreate(row):
    # print(type(row['Product Size']))
    if  isinstance(row['Product Size'], float):
        return None
    else:
        return row['Product Size'][0]
df_BFC = df_BFC.loc[df_BFC['Manufacturing Plant'] == 'B095'].copy()
df_BFC['Date'] = pd.to_datetime(df_BFC['Manufacturing Date']).dt.date
df_BFC.dropna(axis=0, subset=['Manufacturing Date'],inplace=True)
df_BFC['Prod_Line'] = df_BFC.apply(prodLine, axis=1)
df_BFC['Size'] = df_BFC.apply(sizeCreate, axis=1)
df_BFC.dropna(axis=0, subset=['Product Size'],inplace=True)
df_BFC['Size'] = pd.to_numeric(df_BFC['Size'], errors='coerce')
df_BFC['Prod_Line'] = pd.to_numeric(df_BFC['Prod_Line'], errors='coerce')
df_BFC.shape

(110, 53)

In [16]:
df_BFC['Internal Case Id'].value_counts()

GCR16462516    3
GCR15008415    3
GCR14277126    2
GCR16283373    2
GCR14077660    2
              ..
GCR14135718    1
GCR13425231    1
GCR14412926    1
GCR14251676    1
GCR17271833    1
Name: Internal Case Id, Length: 94, dtype: int64

In [17]:
df_merged_final = pd.merge(df_UT,df_BFC[['Date','Size','Prod_Line','Internal Case Id','Contact Summary English1','L3 Comment Code/Comment Description']],how='left',on=['Date','Prod_Line','Size'])

In [18]:
df_merged_final['Internal Case Id'].value_counts()

GCR14135718    25
GCR15690312    25
GCR16172109    24
GCR15204721    23
GCR16574110    23
GCR16912286    23
GCR17068057    23
GCR16716752    16
GCR16595247    16
GCR16997189    10
GCR16247547    10
Name: Internal Case Id, dtype: int64

In [19]:
df_merged_final.to_csv('timeseriesMerge_182.csv')

In [20]:
df_merged_final.columns

Index(['Time', 'Product Description', 'Description', 'Size', 'Prod_Line',
       '10181 DAL Weight CC', '10206 DAL Placement MD CC',
       '10207 CS10 Patch to AQL Placement MD CC',
       '10210 IC Elastic Glue/Pucker Length OP CC',
       '10211 IC Elastic Glue/Pucker Length DS CC',
       '10214 OC Elastic Glue/Pucker Length OP CC',
       '10215 OC Elastic Glue/Pucker Length DS CC',
       '10944 Core Weight Septile 1-4 Front 50% CC',
       '10945 Total Core Weight P1 CC', '10946 Total Core Weight P2 CC',
       '10988 Core Endflap Length UTE P1 CC (Leakage)',
       '10989 Core Endflap Length UTE P2 CC (Leakage)',
       '10990 Core Endflap Length UTE P3 CC (Leakage)',
       '10991 Core Endflap Length UTE P4 CC (Leakage)',
       '11442 CS10 Patch Placement CD CC',
       '12724 Chassis Endflap Length Front CC',
       '12725 IC Elastic Glue/Pucker Plmt Front OP CC',
       '12726 IC Elastic Glue/Pucker Plmt Front DS CC',
       '12727 OC Elastic Glue/Pucker Plmt Back OP CC',
 

In [21]:
import numpy as np
for bol in list(df_merged_final[df_merged_final['Internal Case Id'].notna()]['L3 Comment Code/Comment Description'].notnull()):
    if bol == False:
        print('voilation')

In [22]:
# Just for Validation

# for ind_date in list(df_UT['Time']):
#     #print(len(list(df_LEAKAGE[(df_LEAKAGE['Time'] == ind_date)]['Product Group'].unique())) == 1)
#     print(df_LEAKAGE[(df_LEAKAGE['Time'] == ind_date)]['Product Group'].unique()[0])
#     break

In [23]:
# for ind_date in list_dates:
    
#     temp_times = list(df_LEAKAGE[df_LEAKAGE['Date'] == ind_date]['Time_in_hrs'].unique())
#     temp_vars =  list(df_LEAKAGE[df_LEAKAGE['Date'] == ind_date]['Variable Description'].unique())

#     for var_des in temp_vars:
#         for time in temp_times:
#             if df_LEAKAGE[(df_LEAKAGE['Date'] == ind_date) & (df_LEAKAGE['Variable Description'] == var_des) & (df_LEAKAGE['Time_in_hrs'] == time)].shape[0] == 0:
#                 df_LEAKAGE.append(df_LEAKAGE[(df_LEAKAGE['Date'] == ind_date) & (df_LEAKAGE['Variable Description'] == var_des)].assign(**{'Time_in_hrs': time}), ignore_index=True)
#                 print(df_LEAKAGE[(df_LEAKAGE['Date'] == ind_date) & (df_LEAKAGE['Variable Description'] == var_des)].assign(**{'Time_in_hrs': time}),'\n')
#                 print(df_LEAKAGE.shape,'\n')

